<a href="https://colab.research.google.com/github/dovidfein/dovidfein/blob/main/ECC_on_secp256r1_with_phonebook_(Dovid).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partner practice advanced
Here we implement the protocol using the `secp256r1` curve, allowing us to encrypt larger and more complex messages.

Make a copy of this notebook with your name.

It is important that you generate a random private key the first time. See below for instructions.

## Step 1: Setup

Build the functions. We will use the `secp256r1` curve.

[Curve parameters](https://neuromancer.sk/std/secg/secp256r1#)

In [37]:
import numpy as np
import random

#The curve parameters

a = 0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
# G is given so 'random point' function is not needed here
G = [0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296,
       0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5]


#The ECC curve
def ecc_p(x):
  return (pow(x,3,p) + a*x + b)%p #changed to pow function

#Inverse point wrt group operation
def invP(P):
  return([P[0]%p,-P[1]%p]) #added %p after first coordinate


#Slope
def slope(P,Q):
  if P == Q:
    m = ((3*pow(P[0],2,p) + a)%p * pow(2*P[1],-1,p))%p #added %p after first part
  else:
    m = ((P[1]-Q[1])%p * pow((P[0]-Q[0]),-1,p))%p #added %p after first part
  return m


#Point sum (defining the group operation)
def pointsum(P,Q):
  if P == 0:
    return(Q)
  elif Q == 0:
    return(P)
  elif P == invP(Q):
    return(0)
  else:
    x3 = (pow(slope(P,Q),2,p)-P[0]-Q[0])%p
    y3 = (P[1] + slope(P,Q)*(x3-P[0]))%p
    return [x3,-y3%p]

#Legendre (computing kP using the double-and-add algorithm)
def legendreP(k,P):
  x = P
  y = k
  z = 0
  while y > 0:
    if y%2 == 1:
      y = y-1
      z = pointsum(z,x)
    else:
      y = y/2
      x = pointsum(x,x)
  return(z)


#Convert text to numerical message
def numbmsg(m):
  msg_ascii = [ord(c)- 32 for c in m] #changed to 32
  val = 0
  for i in range(0,len(msg_ascii)):
    val = val*95 + msg_ascii[i] #changed to 95
  return val


#Convert the numerical message to text
def textmsg(m):
  char = ''
  while m > 0:
    char += chr(m%95 + 32)#changed to 95 and 32
    m = m//95#changed to 95
  char = char[::-1]#reverse for the encrypted message
  return(char)

#Find the point that codifies the message
def codify(m):
  x = numbmsg(msg)*95 #adding the @ symbol. changed to 95
  y2 = ecc_p(x)#gives y^2 (in Zp)

  while pow(y2,(p-1)//2,p) != 1:#This has been fixed
    x += 1
    y2 = ecc_p(x)

  root1 = pow(y2,(p+1)//4,p)#One of the roots. This has been fixed

  y = min(root1, p - root1, p)#Takes the smaller of the two roots in Zp
  return [x,y]

## Step 2: Keys

Use the following **only** the first time, when creating the keys. Running it again will require that you share the public key again!!
<br> Copy and paste the output to the following cell as d_A.

In [ ]:
# d_A = random.randint(0,p-1)
# print(hex(d_A)) #Use these two lines the first time to generate the key.

In [38]:
d_A = 0x279538420601f57d1cae3d55ccfefff84e12d0222090380293f7073aadda508a #Keep this secret!!
H_A = legendreP(d_A,G)
print('Your public key to be shared with your partner is H_A.\nx-coordinate: ',
      hex(H_A[0]), '\ny-coordinate: ',hex(H_A[1]))

Your public key to be shared with your partner is H_A.
x-coordinate:  0xd54eba5ed1bfed2c2d6138c3d796770b62f5855625521fd1bd978a3544acd96d 
y-coordinate:  0x5d2e3fd90152a584c567362e2c62d8dbf040738f6530dd266bbb19eccc7d797e


### Import phonebook from Google Sheets
Students all complete the survey using this [link](https://forms.gle/N7Z1jiShxcJ6NzUB8).

The sheet must be stored in the user's drive.

Share link with all students (view only).

The phonebook is stored as a `dataframe` and can be manipulated.

 - It was alphabetized above (leave name as just one field so students can add a last name/initial?).
 - Pull list of names.
 - Print list with numbers alongside. User selects number. If not, re-rerun function.
 - Once number is selected, the corresponding x- and y- coordinates will be saved for the ECC protocol.
 - Messages to be broadcast in group. Only intended recipient can decrypt.

In [46]:
from google.colab import auth
import gspread as gs
from google.auth import default
import pandas as pd

auth.authenticate_user()
creds, _ = default()

gc = gs.authorize(creds)

ws = gc.open('Phonebook (Responses)').sheet1#worksheet

# get_all_values gives a list of rows.
# rows = ws.get_all_values()
# print(rows)
# Convert to a DataFrame and render
# pd.DataFrame.from_records(rows)

df = pd.DataFrame(ws.get_all_records())
df = df.sort_values('Name')#Use ascending = False for reverse order
# df.head()#To see the dataframe.

def select_rec():
  pos = -1
  for name in df.Name:
    pos += 1
    print("For",name, "type",pos)
  receiver = int(input())
  if receiver not in list(range(len(df.Name))):
    print("\nPlease type one of the numbers listed.")
    return select_rec()
  else:
    print("\nYou will be communicating with",df.iloc[receiver,1])
    H_partner = [int(df.iloc[receiver,2],16),int(df.iloc[receiver,3],16)]#Pull the receiver's key from the dataframe.
    return H_partner

H_partner = select_rec()

# print('The public key you asked for is H:\nx-coordinate: ',
#       hex(H_partner[0]), '\ny-coordinate: ',hex(H_partner[1]))#Just to double check that it works. Hide later.

S = legendreP(d_A,H_partner)
# print('The shared secret is S:\nx-coordinate: ',hex(S[0]),'\ny-coordinate: ',hex(S[1]))

For Dovid type 0
For Manuela type 1
For Michele type 2
2

You will be communicating with Michele


## Step 3a: Send an encrypted message

In [41]:
msg = input("Type the message that you wish to send: ")
M = codify(msg)#This codifies the message.
M_prime = pointsum(M,S)#This encrypts the message
print("Send the encrypted message.\n x-coordinate: "
      ,hex(M_prime[0]),"\n y-coordinate: ",hex(M_prime[1]))

Type the message that you wish to send: Don't look:) hehehe
Send the encrypted message.
 x-coordinate:  0x2fb601f42730353748440015cdf7b53dcbf1721047539561937fcc5e1443a38a 
 y-coordinate:  0x73b01676e02a8aaa8b5ff90bc6c91e7de4aa0540c5427e653ce4ada172cd21cc


## Step 3b: Receive an encrypted message and decrypt

In [47]:
enc_x = int(input("Enter the x-coordinate of your encrypted message: "),16)
enc_y = int(input("Enter the y-coordinate of your encrypted message: "),16)
enc = [enc_x,enc_y]
dec_msg = textmsg(pointsum(enc,invP(S))[0])
print('The decrypted message is:',dec_msg.rstrip(dec_msg[-1]))

Enter the x-coordinate of your encrypted message: 0x4dc1133f01b3f299328f1d4bc76805bb87edffc2170e4973ee036744ac752824 
Enter the y-coordinate of your encrypted message: 0xc91b037d6dbfc3da4acf0197ef095245a0da12c536b8de6e69898ef1757677e
The decrypted message is: Ciao Dovid!


# Resources
[Volya Elliptic Curves in Python](https://volya.xyz/ecc/)
<br>
[ECC: a gentle introduction](https://andrea.corbellini.name/2015/05/17/elliptic-curve-cryptography-a-gentle-introduction/)





